In [86]:
import json
import csv

In [96]:
# read converted geojson file as 1
# with open(.geojson) as f
#   json.load(f)

# read income data as 2

# create new list

# iterate through features from #1
# for each_feature in feature_list: 
#   finding corresponding income data
#   for each_income in income_data: 
#       if each_income_city==each_feature_city: 
#           each_feature['properties']['income']
#               new_list.append(each_feature)

# function to create key names from csv data
def nameKey(poll, date):
    return f"{poll}-{date[5:7]}"

# open json
with open("../data/CA_Places.geojson") as f:
    json_data=json.load(f)

    # new list that we'll append to
    poll_cities = []

    # iterate through json data
    for each_feature in json_data["features"]:
        
        # flag to copy to new list - false unless we find income data
        copyStatus = False

        # add income data to feature
        with open("../data/coords.csv") as csv_file:
            income_data = csv.reader(csv_file)
            next(income_data)

            for income in income_data:
                if income[0] == each_feature["properties"]["NAME"]:
                    copyStatus = True
                    each_feature["properties"]["INCOME"] = income[3]

        # add carbon pollution data
        with open("../../../output_data/carbon_monthly.csv") as carbon_file:
            carbon_data = csv.reader(carbon_file)
            next(carbon_data)
            for row in carbon_data:
                if row[1] == each_feature["properties"]["NAME"] and row[3] != "":
                    each_feature["properties"][nameKey("carbon",row[2])] = float(row[3])

        # add nitrogen pollution data
        with open("../../../output_data/nitrogen_monthly.csv") as nitrogen_file:
            nitrogen_data = csv.reader(nitrogen_file)
            next(nitrogen_data)
            for row in nitrogen_data:
                if row[1] == each_feature["properties"]["NAME"] and row[3] != "":
                    each_feature["properties"][nameKey("nitrogen",row[2])] = float(row[3])

        # add ozone pollution data
        with open("../../../output_data/ozone_monthly.csv") as ozone_file:
            ozone_data = csv.reader(ozone_file)
            next(ozone_data)
            for row in ozone_data:
                if row[1] == each_feature["properties"]["NAME"] and row[3] != "":
                    each_feature["properties"][nameKey("ozone",row[2])] = float(row[3])

        # add PM10 pollution data
        with open("../../../output_data/pm10_monthly.csv") as pm10_file:
            pm10_data = csv.reader(pm10_file)
            next(pm10_data)
            for row in pm10_data:
                if row[1] == each_feature["properties"]["NAME"] and row[3] != "":
                    each_feature["properties"][nameKey("pm10",row[2])] = float(row[3])

        # add PM2.5 pollution data
        with open("../../../output_data/pm25_monthly.csv") as pm25_file:
            pm25_data = csv.reader(pm25_file)
            next(pm25_data)
            for row in pm25_data:
                if row[1] == each_feature["properties"]["NAME"] and row[3] != "":
                    each_feature["properties"][nameKey("pm25",row[2])] = float(row[3])
        if copyStatus:
            poll_cities.append(each_feature)

In [97]:
new_json = {}
new_json["type"] = json_data["type"]
new_json["name"] = "CA_Places_filtered"
new_json["crs"] = json_data["crs"]
new_json["features"] = poll_cities

In [98]:
with open("../data/filteredCitiesOrigCoords.geojson", "w") as newFile:
    json.dump(new_json, newFile)


In [101]:

with open("../data/EPA2020_GGasEmissions_LargeFacilities.csv") as emitter_file:
    features_list = []
    emitter_data = csv.reader(emitter_file)
    next(emitter_data)
    for row in emitter_data:
        geometry = {"type": "Point", "coordinates": [float(row[2]), float(row[1])]}
        props = {"name": row[0], "parentCompanies": row[4], "CO2e": int(row[5])}
        features_list.append({"type": "Feature", "geometry": geometry, "properties": props})
    geojson_coll = {"type": "FeatureCollection", "features": features_list}
with open("../data/emitters.geojson", "w") as geo_file:
    json.dump(geojson_coll, geo_file)

In [106]:
# open json
with open("../CA_Places.geojson") as f:
    json_data=json.load(f)

    # new list that we'll append to
    bay_area_geo = []

    # iterate through json data
    for each_feature in json_data["features"]:
        
        # flag to copy to new list - false unless we find income data
        copyStatus = False

        # add income data to feature
        with open("../../../input_data/ACS_income_data_bayarea2020.csv") as csv_file:
            income_data = csv.reader(csv_file)
            next(income_data)

            for income in income_data:
                if income[0] == each_feature["properties"]["NAME"]:
                    copyStatus = True
                    each_feature["properties"]["INCOME"] = float(income[1])

        if copyStatus:
            bay_area_geo.append(each_feature)

income_json = {}
income_json["type"] = json_data["type"]
income_json["name"] = "CA_Places_filtered"
income_json["crs"] = json_data["crs"]
income_json["features"] = bay_area_geo

with open("../data/incomeCitiesOrigCoords.geojson", "w") as newFile:
    json.dump(income_json, newFile)